In [66]:
# Auto-reload setup
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import sys
import os
sys.path.append(os.path.abspath("../src"))
from data_processing import *
print('Successful import')

Successful import


In [68]:
folder = '../data/raw'
types = ['prices_raw', 'total_load_day_ahead_raw', 'generation_production_raw']

grouped = group_files_by_type(folder, types, extension='.csv')
dfs = load_dataframes(grouped)

# Access each DataFrame
df_prices = dfs['prices_raw']
df_load = dfs['total_load_day_ahead_raw']
df_gen = dfs['generation_production_raw']
print(len(df_load))

4


In [69]:
df_load = [drop_unecessary_columns(df, 'Day-ahead Total Load Forecast [MW] - BZN|FR') for df in df_load] 
df_load = [rename_and_reoder_columns(df, None, ['Date', 'Total Load']) for df in df_load]
df_load = [setup_time(df, 'Date', '%d.%m.%Y %H:%M') for df in df_load]
df_load = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Total Load') for df in df_load]
print(len(df_load))
df_summary(df_load[0])
df = concat_data(df_load)
df_summary(df)
df.to_csv('../data/clean/TESST.csv', index=False)

4
Shape: (8784, 2)

Column Types:
Date          datetime64[ns]
Total Load           float64
dtype: object

Missing Values:
Date          0
Total Load    0
dtype: int64

First Rows:
                 Date  Total Load
0 2024-01-01 00:00:00     52847.0
1 2024-01-01 01:00:00     51600.0
2 2024-01-01 02:00:00     50992.0
3 2024-01-01 03:00:00     48187.0
4 2024-01-01 04:00:00     46479.0

Last Rows:
                    Date  Total Load
8779 2024-12-31 19:00:00     68220.0
8780 2024-12-31 20:00:00     65266.0
8781 2024-12-31 21:00:00     62886.0
8782 2024-12-31 22:00:00     63983.0
8783 2024-12-31 23:00:00     64603.0
----------------------------------------
OUI
OUI
OUI
OUI
Shape: (35064, 2)

Column Types:
Date          datetime64[ns]
Total Load           float64
dtype: object

Missing Values:
Date          0
Total Load    0
dtype: int64

First Rows:
                 Date  Total Load
0 2021-01-01 00:00:00     65761.0
1 2021-01-01 01:00:00     64139.0
2 2021-01-01 02:00:00     62657.0
3 2021-0

/Users/gonot/Desktop/Energy project/epex-spot-analysis/src/data_processing.py:54: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').mean()
/Users/gonot/Desktop/Energy project/epex-spot-analysis/src/data_processing.py:54: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').mean()
/Users/gonot/Desktop/Energy project/epex-spot-analysis/src/data_processing.py:54: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').mean()
/Users/gonot/Desktop/Energy project/epex-spot-analysis/src/data_processing.py:54: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').mean()


In [70]:
df = load_epex_data('../data/raw/2021_prices_raw.csv')
df = clean_data(df, 'MTU (CET/CEST)')
df.to_csv('../data/clean/2021_prices_clean.csv', index=False)

NameError: name 'clean_data' is not defined

In [ ]:
from data_processing import exploitable_data

folderpath = '../data/clean'
df_concat = exploitable_data(folderpath)
df_concat.to_csv(f'{folderpath}/exploitable/multiple_years.csv', index=False)